# Interactive Rink
*Ce notebook nous permet de jouer avec des sliders interactifs qui changent l'affichage d'un graphique qui résume très brievement les évenements et les affiche sur la glace.*

In [3]:
from ift6758.data import play_context, play_description, get_game
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import clear_output

In [4]:
PROJECT_ROOT = Path.cwd().parent  # notebooks -> Project-Hockey
RINK_PATH = PROJECT_ROOT / "figures" / "nhl_rink.png"

In [5]:
def draw_play(index=0):
    """Affiche les informations d'un jeu ainsi qu'un diagramme avec l'endroit du jeu 
    Méthode utilisée pour le slider play_ui

    Paramètre:
    index (int): valeur initiale pour le IntSlider
    """
    clear_output(wait=True)
    
    # Création de variables pour l'affichage
    current_play = all_plays[index]
    details = current_play.get('details', {}) or {}
    x = details.get('xCoord')
    y = details.get('yCoord')
    start_utc = data.get("startTimeUTC", "")
    game_id = data.get("id", "")
    last_type = (data.get("gameOutcome") or {}).get("lastPeriodType", "")
    last_type = {"REG": "", "OT": "OT", "SO": "SO"}.get(last_type, last_type)
    period = (current_play.get("periodDescriptor") or {}).get("number", "?")
    time_in_period = current_play.get("timeInPeriod", "??:??")
    desc = play_description(current_play, team_by_id, player_name)

    # Affichage des informations du jeu
    print(start_utc)
    print(f"Game ID: {game_id}; {home['abbrev']} (home) vs {away['abbrev']} (away)")
    if last_type:
        print(last_type)
    print("")
    print(f"{'':12}{'Home':>6}{'Away':>6}")
    print(f"{'Teams:':12}{home['abbrev']:>6}{away['abbrev']:>6}")
    print(f"{'Goals:':12}{home.get('score','?'):>6}{away.get('score','?'):>6}")
    print(f"{'SoG:':12}{home.get('sog','?'):>6}{away.get('sog','?'):>6}")
    print(f"{'SO Goals:':12}{'None':>6}{'None':>6}")
    print(f"{'SO Attempts:':12}{'None':>6}{'None':>6}")
    print("")

    # Dessin du diagramme du jeu
    img = np.asarray(Image.open(RINK_PATH).convert('RGB'))
    my_extent = [-100, 100, -42.5, 42.5] 
    plt.imshow(img, extent=my_extent)
    plt.text(-60, 45, away["abbrev"], ha="center", va="bottom")
    plt.text(60, 45, home["abbrev"], ha="center", va="bottom")
    if x is not None and y is not None:
        plt.scatter(x, y, s=100, c='blue', marker='o')   
    plt.title(f"{desc}\n{time_in_period} P-{period}")  
    plt.show()

def on_game_change(change):
    """ Méthode utilisée pour le slider game_slider
    """
    global data, all_plays, home, away, team_by_id, player_name
    new_game = change["new"]
    data = get_game(year, game_type, new_game)
    all_plays = data.get("plays", []) or []
    home, away, team_by_id, player_name = play_context(data)
    idx_slider.max = max(0, len(all_plays) - 1)

    # Si le slider est à 0, on force le rafraichissement de play_ui
    if idx_slider.value == 0:
        output_widget = play_ui.children[-1]
        output_widget.clear_output(wait=True)
        with output_widget:
            draw_play(0)  
    # Sinon on le met à 0 (rafraichit par lui-même)
    else:
        idx_slider.value = 0

In [6]:
global all_plays, data, home, away, team_by_id, player_name

year = 2023
game_type = 2
data = get_game(year, game_type, 1)
all_plays = data["plays"]
home, away, team_by_id, player_name = play_context(data)

game_slider = widgets.IntSlider(
    value=1, min=1, max=1312, step=1, description="Game",
    continuous_update=False
)
play_ui = widgets.interactive(draw_play, index=(0, len(all_plays)-1, 1)) 
idx_slider = play_ui.children[0]  
game_slider.observe(on_game_change, names="value")

display(widgets.VBox([game_slider, play_ui]))